<a href="https://colab.research.google.com/github/dimanyen/card-benefits/blob/main/%E4%BF%A1%E7%94%A8%E5%8D%A1%E5%84%AA%E6%83%A0GAI_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package

In [ ]:
!pip install langchain chromadb openai

# 建立向量資料庫

In [5]:
! unzip /content/信用卡優惠訊息-20240719T073550Z-001.zip

Archive:  /content/信用卡優惠訊息-20240719T073550Z-001.zip
  inflating: 信用卡優惠訊息/星展銀行.pdf  
  inflating: 信用卡優惠訊息/信用卡優惠懶人包(保費、燃料費、王品、首刷禮、機場貴賓室).pdf  
  inflating: 信用卡優惠訊息/匯豐銀行.pdf  
  inflating: 信用卡優惠訊息/中國信託.pdf  
  inflating: 信用卡優惠訊息/玉山銀行.pdf  
  inflating: 信用卡優惠訊息/台新銀行.pdf  
  inflating: 信用卡優惠訊息/國泰世華.pdf  


In [6]:
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from google.colab import drive
from langchain.docstore.document import Document
import chromadb
from chromadb.config import Settings
from google.colab import userdata

api_key = userdata.get('OPENAI_API_KEY')

# 掛載Google Drive
# drive.mount('/content/drive', force_remount=True)

# 設置資料夾路徑
folder_path = "/content/信用卡優惠訊息"

# 初始化PDF文件加載器
pdf_loaders = []

# 遍歷資料夾內所有PDF文件
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        pdf_loaders.append(PyPDFLoader(file_path))

# 確認有無PDF文件
if not pdf_loaders:
    raise FileNotFoundError(f"No PDF files found in directory: {folder_path}")

# 加載所有PDF文件
documents = []
for loader in pdf_loaders:
    documents.extend(loader.load())

# 確認有無加載文檔
if not documents:
    raise ValueError("No documents were loaded.")

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=30
)
texts = text_splitter.split_documents(documents)

# 確認有無分割文本
if not texts:
    raise ValueError("Text splitting failed.")

# 初始化OpenAI嵌入
embeddings = OpenAIEmbeddings(api_key=api_key)

# 提取文本內容
document_texts = [doc.page_content for doc in texts]

# 確認有無文本內容
if not document_texts:
    raise ValueError("No text content found in documents.")

# 嵌入文檔
document_embeddings = embeddings.embed_documents(document_texts)

# 確認有無嵌入文檔
if not document_embeddings:
    raise ValueError("Document embedding failed.")

# 初始化Chroma客戶端
chroma_client = chromadb.Client(Settings(persist_directory="/content/drive/MyDrive/vector_store"))

# 創建Chroma集合
collection = chroma_client.create_collection(name="document_collection")

# 添加文本和對應的嵌入到Chroma集合
for i, (embedding, text) in enumerate(zip(document_embeddings, document_texts)):
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        metadatas=[{"text": text}]
    )

print("Documents have been successfully added to the Chroma vector store.")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Documents have been successfully added to the Chroma vector store.


In [7]:
query = "我想要去日本玩，有麼優惠推薦"
# 將查詢轉換為嵌入向量
query_embedding = embeddings.embed_query(query)

# 在Chroma中搜索相似文檔
results = collection.query(query_embeddings=[query_embedding], n_results=5)

# 獲取相似文檔的文本
similar_texts = [metadata["text"] for metadata in results["metadatas"][0]]

display(similar_texts)

['商家：國內一般消費  \n詳情：每筆國內一般消費不限金額可享 1%現金回饋，回饋無上限，直接折抵次\n期帳單消費。  \n#現金回饋最高 3%，無上限 (2024/12/31 止) \n商家：日本  \n詳情：使用玉山 JCB日圓極緻卡於日本地區消費享 1.5%現金回饋，同享免收\n1.5%國外交易服務費，合計共 3%回饋。  \n#指定房型 43折(2024/12/31 止、限平日 ) \n商家：台中商旅  \n詳情：入住菁英套房、景觀套房依定價享平日 4.9折、假日 5.4折優惠，行政套',
 '4.紅檜俱樂部 +竹林亭單饗餐 (日式套餐，原價 NT$1,080+10%) NT$1,600/ 人。  \n條件：  \n限平日使用，限泡湯  \n--- \n##台新銀行 @GoGo悠遊御璽卡  \n#台新 Point(信用卡 )回饋最高 0.3%，無上限 (2024/11/30 止) \n商家：國內一般消費、海外一般消費  \n詳情：一般消費  0.3%無上限。  \n#餐飲 5折(2024/8/31 止) \n商家： FunNow、日本',
 '詳情：國內一般消費享 1%現金回饋無上限。  \n#餐飲 5折(2024/8/31 止) \n商家： FunNow、日本  \n詳情：台灣發行之 Visa無限  /商務無限 / 御璽 /商務御璽卡持卡人，預訂東京、\n大阪、沖繩等日本地區指定精選餐廳，透過 FunNow訂位，訂位人數選擇  2人\n2份套餐，於付款頁面輸入優惠碼「 Visa50off 」後，可立即享有線上預訂金額\n50％ 折扣，每月限量 100名。  \n※優惠實際適用餐廳請依 FunNow活動網頁為準。  \n條件：限餐飲',
 '日、國定假日 )使用原價 7折。  \n3.紅檜俱樂部 NT$1,200/ 人。  \n4.紅檜俱樂部 +竹林亭單饗餐 (日式套餐，原價 NT$1,080+10%) NT$1,600 /人。  \n條件：限平日使用，限泡湯  \n--- \n##台新銀行玫瑰 Giving卡 \n#台新 Point(信用卡 )回饋最高 3%，每月上限台新 Point(信用卡 )回饋 3000點\n(2024/12/31 止、限國定例假日 ) \n商家：國內一般消費',
 '商家： FunNow、日本  \n詳情：台灣發行之 Visa無限

In [8]:
from openai import OpenAI
from IPython.display import Markdown, display

client = OpenAI(api_key=api_key)
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "根據我輸入的問題與內容生成結果"},
    {"role": "user", "content": f'問題:{query}\n內容:{similar_texts}'}
  ]
)


display(Markdown(completion.choices[0].message.content))

如果您計劃去日本旅遊，這裡有一些值得考慮的優惠推薦：

1. **玉山 JCB日圓極緻卡**：
   - **優惠詳情**：在日本地區消費可享 1.5% 現金回饋，還免收 1.5% 國外交易服務費，合計可享 3% 回饋。
   - **優惠期限**：截至 2024/12/31。

2. **透過 FunNow 訂位**：
   - **優惠詳情**：持有台灣發行之 Visa無限、商務無限、御璽或商務御璽卡的持卡人，可以預訂東京、大阪、沖繩等指定餐廳。訂位 2 人並選擇 2 份套餐的情況下，在付款頁面輸入優惠碼「Visa50off」，可享受訂位金額 50% 折扣。
   - **限制條件**：每月限量 100 名。
   - **優惠期限**：截至 2024/8/31。

3. **特定餐廳餐飲優惠**：
   - **優惠詳情**：透過 FunNow 訂位，在指定餐廳可享受餐點買一送一優惠。
   - **優惠期限**：截至 2024/12/31。

這些優惠可以幫助您在日本旅遊時節省不少費用，建議您根據實際需要選擇並確認參考優惠的具體條件和期限。祝您有個愉快的日本之旅！